<a href="https://colab.research.google.com/github/konerjonlar/Akbank-Makine-renmesi-Bootcamp/blob/main/Housing_Price_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Required Libraries

In [2]:
# for data analysis
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# for modellling and evaluating performance of the model
from sklearn import model_selection
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor


# Project Definition

For this project, we need to load the Melbourne Housing dataset into our project. The
quality and amount of data we collect will determine how good our predictive model
can be. For this reason, we need to examine the dataset very carefully. We will
estimate the price of a house using the Melbourne Housing dataset, which is a
real-life example. Before evaluating any cost, we will start by analyzing the data
using preprocessing techniques. We will then build our models and measure their
performance to complete the project.

# Gathering and Observing Data

In [31]:
# Load dataset
df = pd.read_csv('Melbourne_housing_FULL.csv')

# Observe the 5 columns
print(df.iloc[:, :5].head())

# Find the shape, number of columns and size of the dataset
print("\nShape of the Dataset:\n", df.shape,
"\nNumber of Columns:\n", len(df.columns),
"\nSize of the Dataset:\n", df.size)


# Display general information about the dataset
print("\nColumn Labels:", df.columns.tolist(),
      "\nColumn Data Types:", df.dtypes,
      "\nMemory Usage:", df.memory_usage(),
      "\nRange Index:", df.index,
      "\nNumber of cells in each column (non-null values):", df.count(), sep="\n")

       Suburb             Address  Rooms Type      Price
0  Abbotsford       68 Studley St      2    h        NaN
1  Abbotsford        85 Turner St      2    h  1480000.0
2  Abbotsford     25 Bloomburg St      2    h  1035000.0
3  Abbotsford  18/659 Victoria St      3    u        NaN
4  Abbotsford        5 Charles St      3    h  1465000.0

Shape of the Dataset:
 (34857, 21) 
Number of Columns:
 21 
Size of the Dataset:
 731997

Column Labels:
['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG', 'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude', 'Longtitude', 'Regionname', 'Propertycount']

Column Data Types:
Suburb            object
Address           object
Rooms              int64
Type              object
Price            float64
Method            object
SellerG           object
Date              object
Distance         float64
Postcode         float64
Bedroom2         float64
Bathroom   

# Exploratory Data Analysis